In [11]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, log_loss

# =======================================
# 1. 데이터 불러오기 및 기본 설정
# =======================================
train = pd.read_csv('train_data_진짜.csv', encoding='cp949')
test = pd.read_csv('test_data_진짜.csv', encoding='cp949')

# 사용할 피처 및 타겟
features = ["난자 점수", "정자 점수", "나이 점수", "시술점수"]
target = "임신 성공 여부"

# ※ 이미 train, test에는 "시술유형매핑" 칼럼이 존재한다고 가정합니다.

# =======================================
# 2. 교차검증 및 early stopping을 적용한 모델 학습 함수
# =======================================
def train_model_cv(X, y, n_splits=5, random_state=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    oof_preds = np.zeros(len(y))
    fold_metrics = []
    best_models = []  # 각 fold의 모델 저장
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        train_data_lgb = lgb.Dataset(X_train, label=y_train)
        val_data_lgb = lgb.Dataset(X_val, label=y_val, reference=train_data_lgb)
        
        params = {
            'objective': 'binary',
            'metric': 'auc',
            'boosting_type': 'gbdt',
            'learning_rate': 0.05,
            'verbose': -1,
            'seed': random_state
        }
        
        # early stopping과 평가 로깅을 위한 콜백(callbacks)
        callbacks = [lgb.early_stopping(stopping_rounds=50, verbose=True),
                     lgb.log_evaluation(period=100)]
        
        clf = lgb.train(
            params,
            train_data_lgb,
            num_boost_round=1000,
            valid_sets=[train_data_lgb, val_data_lgb],
            valid_names=['train', 'valid'],
            callbacks=callbacks
        )
        
        val_pred = clf.predict(X_val, num_iteration=clf.best_iteration)
        oof_preds[val_idx] = val_pred
        best_models.append(clf)
        
        val_pred_binary = (val_pred >= 0.5).astype(int)
        acc = accuracy_score(y_val, val_pred_binary)
        auc = roc_auc_score(y_val, val_pred)
        f1 = f1_score(y_val, val_pred_binary)
        ll = log_loss(y_val, val_pred)
        fold_metrics.append({'fold': fold+1, 'accuracy': acc, 'auc': auc, 'f1': f1, 'log_loss': ll})
        print(f"Fold {fold+1}: Accuracy: {acc:.4f}, AUC: {auc:.4f}, F1: {f1:.4f}, LogLoss: {ll:.4f}")
        
    overall_acc = accuracy_score(y, (oof_preds >= 0.5).astype(int))
    overall_auc = roc_auc_score(y, oof_preds)
    overall_f1 = f1_score(y, (oof_preds >= 0.5).astype(int))
    overall_ll = log_loss(y, oof_preds)
    print(f"Overall CV: Accuracy: {overall_acc:.4f}, AUC: {overall_auc:.4f}, F1: {overall_f1:.4f}, LogLoss: {overall_ll:.4f}")
    
    # 각 fold의 모델 리스트, fold별 metric, 전체 metric 반환
    return best_models, fold_metrics, {'accuracy': overall_acc, 'auc': overall_auc, 'f1': overall_f1, 'log_loss': overall_ll}

# =======================================
# 3. Train 데이터에서 모델 학습 (DI와 IVF 별로)
# =======================================
models = {}     # 학습된 모델(각 fold 모델 리스트) 저장. 키: 'DI', 'IVF_0', 'IVF_1', 'IVF_2'
cv_metrics = {} # 교차검증 성능 저장

# (A) DI 모델 (시술 유형 == "DI")
di_train = train[train["시술 유형"] == "DI"].copy()
if len(di_train) > 0:
    X_di = di_train[features]
    y_di = di_train[target].astype(int)
    print("=== Training DI 모델 ===")
    di_models, di_fold_metrics, di_overall_metrics = train_model_cv(X_di, y_di)
    models["DI"] = di_models
    cv_metrics["DI"] = di_fold_metrics

# (B) IVF 모델 (시술 유형 == "IVF") - "시술유형매핑" 값별로 0, 1, 2
ivf_train = train[train["시술 유형"] == "IVF"].copy()
for mapping_value in [0, 1, 2]:
    subset = ivf_train[ivf_train["시술유형매핑"] == mapping_value].copy()
    if len(subset) > 0:
        X_ivf = subset[features]
        y_ivf = subset[target].astype(int)
        print(f"=== Training IVF 모델 for 시술유형매핑 = {mapping_value} ===")
        ivf_models, ivf_fold_metrics, ivf_overall_metrics = train_model_cv(X_ivf, y_ivf)
        models[f"IVF_{mapping_value}"] = ivf_models
        cv_metrics[f"IVF_{mapping_value}"] = ivf_fold_metrics

print("=== 교차검증 결과 ===")
print(cv_metrics)

=== Training DI 모델 ===
Training until validation scores don't improve for 50 rounds
[100]	train's auc: 0.724284	valid's auc: 0.574263
Early stopping, best iteration is:
[148]	train's auc: 0.731185	valid's auc: 0.575526
Fold 1: Accuracy: 0.8689, AUC: 0.5755, F1: 0.0984, LogLoss: 0.3903
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[14]	train's auc: 0.683005	valid's auc: 0.644496
Fold 2: Accuracy: 0.8736, AUC: 0.6445, F1: 0.0479, LogLoss: 0.3670
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[4]	train's auc: 0.671216	valid's auc: 0.647622
Fold 3: Accuracy: 0.8712, AUC: 0.6476, F1: 0.0000, LogLoss: 0.3764
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	train's auc: 0.677579	valid's auc: 0.612744
Fold 4: Accuracy: 0.8712, AUC: 0.6127, F1: 0.0000, LogLoss: 0.3822
Training until validation scores don't improve for 50 rounds
Early stopping, bes

In [17]:
# =======================================
# 4. Test 데이터에 대한 예측 (학습한 모델 재사용)
# =======================================
# test 데이터의 "임신 성공 여부" 칼럼을 채워서 저장합니다.
test["임신 성공 여부"] = np.nan

# 예측 시 각 fold의 모델 예측을 평균내는 앙상블 함수 정의
def predict_ensemble(models_list, X):
    preds = np.zeros(X.shape[0])
    for mdl in models_list:
        preds += mdl.predict(X, num_iteration=mdl.best_iteration)
    preds /= len(models_list)
    return preds

# (A) DI 예측: "시술 유형"이 DI인 경우
di_test = test[test["시술 유형"] == "DI"].copy()
if len(di_test) > 0:
    X_di_test = di_test[features]
    pred_di = predict_ensemble(models["DI"], X_di_test)
    test.loc[di_test.index, "임신 성공 여부"] = (pred_di >= 0.5).astype(int)

# (B) IVF 예측: "시술 유형"이 IVF인 경우
ivf_test = test[test["시술 유형"] == "IVF"].copy()
if len(ivf_test) > 0:
    # test 데이터에는 이미 "시술유형매핑" 칼럼이 존재한다고 가정합니다.
    for mapping_value in [0, 1, 2]:
        subset_idx = test[(test["시술 유형"] == "IVF") & (test["시술유형매핑"] == mapping_value)].index
        if len(subset_idx) > 0 and f"IVF_{mapping_value}" in models:
            X_ivf_test = test.loc[subset_idx, features]
            pred_ivf = predict_ensemble(models[f"IVF_{mapping_value}"], X_ivf_test)
            test.loc[subset_idx, "임신 성공 여부"] = (pred_ivf >= 0.5).astype(int)

# =======================================
# 5. 예측 결과 저장 및 요약 출력
# =======================================
test.to_csv("test_with_predictions2.csv", index=False, encoding = 'cp949')
print("최종 예측 결과가 'test_with_predictions.csv' 파일에 저장되었습니다.")

최종 예측 결과가 'test_with_predictions.csv' 파일에 저장되었습니다.


In [25]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss

# =======================================
# 1. 데이터 불러오기 및 기본 설정
# =======================================
train = pd.read_csv('train_data_진짜.csv', encoding='cp949')
test = pd.read_csv('test_data_진짜.csv', encoding='cp949')

# 사용할 피처 및 타겟
features = ["난자 점수", "정자 점수", "나이 점수", "시술점수"]
target = "임신 성공 여부"

# ※ train, test에는 이미 "시술유형매핑"과 "시술 유형" 칼럼이 존재한다고 가정합니다.

# =======================================
# 2. 스태킹 앙상블 모델 하이퍼파라미터 튜닝 함수
# =======================================
def tune_stack_model(X, y, random_state=42):
    """
    스태킹 앙상블 모델(기본 추정기: LGBM, XGB, RF, CatBoost; 최종 추정기: LogisticRegression)을 
    GridSearchCV로 하이퍼파라미터 튜닝하여 최적 모델을 반환합니다.
    """
    base_estimators = [
        ('lgb', lgb.LGBMClassifier(random_state=random_state)),
        ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=random_state)),
        ('rf', RandomForestClassifier(random_state=random_state)),
        ('cat', CatBoostClassifier(verbose=0, random_state=random_state))
    ]
    
    stacking = StackingClassifier(
        estimators=base_estimators,
        final_estimator=LogisticRegression(),
        passthrough=True,
        cv=5,
        n_jobs=-1
    )
    
    param_grid = {
        'lgb__learning_rate': [0.01, 0.05, 0.1],
        'lgb__n_estimators': [100, 300],
        'xgb__learning_rate': [0.01, 0.05, 0.1],
        'xgb__n_estimators': [100, 300],
        'rf__n_estimators': [100, 200],
        'cat__iterations': [100, 200],
        'final_estimator__C': [0.1, 1.0, 10.0]
    }
    
    grid = GridSearchCV(estimator=stacking, param_grid=param_grid, cv=5, scoring='roc_auc', 
                        n_jobs=-1, verbose=1)
    grid.fit(X, y)
    print("Best parameters: ", grid.best_params_)
    print("Best CV score: ", grid.best_score_)
    return grid.best_estimator_

# =======================================
# 3. Train 데이터에서 그룹별 스태킹 모델 튜닝 및 학습
# =======================================
stack_models = {}  # 각 그룹별 최적 모델 저장. 키: 'DI', 'IVF_0', 'IVF_1', 'IVF_2'

# (A) DI 그룹: "시술 유형" == "DI"
di_train = train[train["시술 유형"] == "DI"].copy()
if len(di_train) > 0:
    X_di = di_train[features]
    y_di = di_train[target].astype(int)
    print("Tuning stacking model for DI...")
    best_model_di = tune_stack_model(X_di, y_di)
    stack_models["DI"] = best_model_di

# (B) IVF 그룹: "시술 유형" == "IVF" – 각 "시술유형매핑" 값별 (0, 1, 2)
ivf_train = train[train["시술 유형"] == "IVF"].copy()
for mapping_value in [0, 1, 2]:
    subset = ivf_train[ivf_train["시술유형매핑"] == mapping_value].copy()
    if len(subset) > 0:
        X_ivf = subset[features]
        y_ivf = subset[target].astype(int)
        print(f"Tuning stacking model for IVF group with 시술유형매핑 = {mapping_value} ...")
        best_model_ivf = tune_stack_model(X_ivf, y_ivf)
        stack_models[f"IVF_{mapping_value}"] = best_model_ivf

# =======================================
# 4. Test 데이터에 대한 예측 (학습한 최적 모델 재사용)
# =======================================
test[target] = np.nan  # 임신 성공 여부 초기화

def predict_best(model, X):
    return model.predict_proba(X)[:, 1]

# (A) DI 예측
di_test = test[test["시술 유형"] == "DI"].copy()
if len(di_test) > 0:
    X_di_test = di_test[features]
    pred_di = predict_best(stack_models["DI"], X_di_test)
    test.loc[di_test.index, target] = (pred_di >= 0.5).astype(int)

# (B) IVF 예측: 각 "시술유형매핑" 값별로 예측 후 할당
ivf_test = test[test["시술 유형"] == "IVF"].copy()
if len(ivf_test) > 0:
    for mapping_value in [0, 1, 2]:
        idx = test[(test["시술 유형"] == "IVF") & (test["시술유형매핑"] == mapping_value)].index
        if len(idx) > 0 and f"IVF_{mapping_value}" in stack_models:
            X_ivf_test = test.loc[idx, features]
            pred_ivf = predict_best(stack_models[f"IVF_{mapping_value}"], X_ivf_test)
            test.loc[idx, target] = (pred_ivf >= 0.5).astype(int)

# =======================================
# 5. 예측 결과 저장 및 출력
# =======================================
test.to_csv("test_with_predictions.csv", index=False)
print("최종 예측 결과가 'test_with_predictions.csv' 파일에 저장되었습니다.")


Tuning stacking model for DI...
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best parameters:  {'cat__iterations': 100, 'final_estimator__C': 10.0, 'lgb__learning_rate': 0.05, 'lgb__n_estimators': 100, 'rf__n_estimators': 200, 'xgb__learning_rate': 0.05, 'xgb__n_estimators': 100}
Best CV score:  0.6495490787871148
Tuning stacking model for IVF group with 시술유형매핑 = 0 ...
Fitting 5 folds for each of 432 candidates, totalling 2160 fits


KeyboardInterrupt: 

In [27]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss

# =======================================
# 1. 데이터 불러오기 및 기본 설정
# =======================================
train = pd.read_csv('train_data_진짜.csv', encoding='cp949')
test = pd.read_csv('test_data_진짜.csv', encoding='cp949')

# 사용할 피처 및 타겟
features = ["난자 점수", "정자 점수", "나이 점수", "시술점수"]
target = "임신 성공 여부"

# ※ train, test에는 이미 "시술유형매핑"과 "시술 유형" 칼럼이 존재한다고 가정합니다.

# =======================================
# 2. CatBoost 하이퍼파라미터 튜닝 함수
# =======================================
def tune_catboost(X, y, random_state=42):
    """
    CatBoostClassifier의 하이퍼파라미터 그리드를 GridSearchCV로 튜닝하여 최적의 모델을 반환합니다.
    """
    model = CatBoostClassifier(
        verbose=0,
        random_state=random_state,
        loss_function='Logloss'
    )
    param_grid = {
        'iterations': [200, 300, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'depth': [4, 6, 8],
        'l2_leaf_reg': [1, 3, 5]
        # 추가 파라미터 (예: border_count)도 필요에 따라 추가 가능
    }
    
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=3,  # 데이터가 크므로 3-fold CV 사용
        scoring='roc_auc',
        n_jobs=-1,
        verbose=1
    )
    grid_search.fit(X, y)
    print("Best parameters: ", grid_search.best_params_)
    print("Best CV score: ", grid_search.best_score_)
    return grid_search.best_estimator_

# =======================================
# 3. Train 데이터에서 그룹별 CatBoost 모델 튜닝 및 학습
# =======================================
catboost_models = {}  # 각 그룹별 최적 모델 저장. 키: 'DI', 'IVF_0', 'IVF_1', 'IVF_2'

# (A) DI 그룹: "시술 유형" == "DI"
di_train = train[train["시술 유형"] == "DI"].copy()
if len(di_train) > 0:
    X_di = di_train[features]
    y_di = di_train[target].astype(int)
    print("Tuning CatBoost model for DI group...")
    best_model_di = tune_catboost(X_di, y_di)
    catboost_models["DI"] = best_model_di

# (B) IVF 그룹: "시술 유형" == "IVF" – 각 "시술유형매핑" 값별 (0, 1, 2)
ivf_train = train[train["시술 유형"] == "IVF"].copy()
for mapping_value in [0, 1, 2]:
    subset = ivf_train[ivf_train["시술유형매핑"] == mapping_value].copy()
    if len(subset) > 0:
        X_ivf = subset[features]
        y_ivf = subset[target].astype(int)
        print(f"Tuning CatBoost model for IVF group with 시술유형매핑 = {mapping_value} ...")
        best_model_ivf = tune_catboost(X_ivf, y_ivf)
        catboost_models[f"IVF_{mapping_value}"] = best_model_ivf

# =======================================
# 4. Test 데이터에 대한 예측 (학습한 최적 모델 재사용)
# =======================================
test[target] = np.nan  # "임신 성공 여부" 초기화

def predict_model(model, X):
    # 예측 확률을 반환
    return model.predict_proba(X)[:, 1]

# (A) DI 예측: "시술 유형"이 DI인 경우
di_test = test[test["시술 유형"] == "DI"].copy()
if len(di_test) > 0:
    X_di_test = di_test[features]
    pred_di = predict_model(catboost_models["DI"], X_di_test)
    test.loc[di_test.index, target] = (pred_di >= 0.5).astype(int)

# (B) IVF 예측: "시술 유형"이 IVF인 경우, 각 "시술유형매핑" 값별로 예측 후 할당
ivf_test = test[test["시술 유형"] == "IVF"].copy()
if len(ivf_test) > 0:
    for mapping_value in [0, 1, 2]:
        idx = test[(test["시술 유형"] == "IVF") & (test["시술유형매핑"] == mapping_value)].index
        if len(idx) > 0 and f"IVF_{mapping_value}" in catboost_models:
            X_ivf_test = test.loc[idx, features]
            pred_ivf = predict_model(catboost_models[f"IVF_{mapping_value}"], X_ivf_test)
            test.loc[idx, target] = (pred_ivf >= 0.5).astype(int)

# =======================================
# 5. 예측 결과 저장 및 출력
# =======================================
test.to_csv("test_with_predictions_catboost3.csv", index=False)
print("최종 예측 결과가 'test_with_predictions_catboost.csv' 파일에 저장되었습니다.")

Tuning CatBoost model for DI group...
Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters:  {'depth': 4, 'iterations': 200, 'l2_leaf_reg': 1, 'learning_rate': 0.01}
Best CV score:  0.6439742917470429
Tuning CatBoost model for IVF group with 시술유형매핑 = 0 ...
Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters:  {'depth': 4, 'iterations': 300, 'l2_leaf_reg': 5, 'learning_rate': 0.05}
Best CV score:  0.8542731065161675
Tuning CatBoost model for IVF group with 시술유형매핑 = 1 ...
Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters:  {'depth': 8, 'iterations': 500, 'l2_leaf_reg': 5, 'learning_rate': 0.01}
Best CV score:  0.8502491369666973
Tuning CatBoost model for IVF group with 시술유형매핑 = 2 ...
Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters:  {'depth': 4, 'iterations': 200, 'l2_leaf_reg': 1, 'learning_rate': 0.05}
Best CV score:  0.660800738966908
최종 예측 결과가 'test_with_predictions_catboost.c

In [29]:
test.to_csv("test_with_predictions_catboost3.csv", index=False, encoding= 'cp949')
print("최종 예측 결과가 'test_with_predictions_catboost.csv' 파일에 저장되었습니다.")

최종 예측 결과가 'test_with_predictions_catboost.csv' 파일에 저장되었습니다.


In [31]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, log_loss

# =======================================
# 1. 데이터 불러오기 및 기본 설정
# =======================================
# 파일 인코딩과 경로는 상황에 맞게 조정하세요.
train = pd.read_csv('train_data_진짜.csv', encoding='cp949')
test = pd.read_csv('test_data_진짜.csv', encoding='cp949')

# 사용할 피처 및 타겟 (칼럼명에 주의)
features = ["난자 점수", "정자 점수", "나이 점수", "시술점수"]
target = "임신 성공 여부"

# 타겟은 정수형으로 변환
train[target] = train[target].astype(int)

# train, test에는 이미 "시술 유형"과 "시술유형매핑" 칼럼이 존재한다고 가정합니다.

# =======================================
# 2. 모델 구축 함수 (딥러닝 네트워크)
# =======================================
def build_model(input_dim, dropout_rate=0.3, l2_reg=0.001):
    model = Sequential()
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(l2_reg), input_dim=input_dim))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(16, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# =======================================
# 3. 각 그룹별 모델 학습 (딥러닝)
# =======================================
# 모델들을 저장할 딕셔너리 (키: "DI", "IVF_0", "IVF_1", "IVF_2")
models = {}
results = {}  # 각 그룹별 validation 결과를 저장

# 공통 학습 파라미터
EPOCHS = 100
BATCH_SIZE = 256
PATIENCE = 10

# (A) DI 그룹: "시술 유형" == "DI"
di_train = train[train["시술 유형"] == "DI"].copy()
if len(di_train) > 0:
    X_di = di_train[features].values
    y_di = di_train[target].values
    # stratified split (80% train, 20% validation)
    X_train_di, X_val_di, y_train_di, y_val_di = train_test_split(X_di, y_di, test_size=0.2, stratify=y_di, random_state=42)
    
    model_di = build_model(input_dim=len(features), dropout_rate=0.3, l2_reg=0.001)
    early_stop = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True, verbose=1)
    
    print("Training DI model...")
    history_di = model_di.fit(X_train_di, y_train_di, 
                              validation_data=(X_val_di, y_val_di),
                              epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stop], verbose=2)
    
    # 예측 및 평가 (Validation)
    val_pred_di = model_di.predict(X_val_di).flatten()
    val_pred_di_bin = (val_pred_di >= 0.5).astype(int)
    acc_di = accuracy_score(y_val_di, val_pred_di_bin)
    auc_di = roc_auc_score(y_val_di, val_pred_di)
    f1_di = f1_score(y_val_di, val_pred_di_bin)
    ll_di = log_loss(y_val_di, val_pred_di)
    print("DI group validation results:")
    print("  Accuracy: {:.4f}".format(acc_di))
    print("  AUC: {:.4f}".format(auc_di))
    print("  F1 Score: {:.4f}".format(f1_di))
    print("  Log Loss: {:.4f}".format(ll_di))
    
    models["DI"] = model_di
    results["DI"] = {"accuracy": acc_di, "auc": auc_di, "f1": f1_di, "log_loss": ll_di}

# (B) IVF 그룹: "시술 유형" == "IVF" – 그룹별: "시술유형매핑" = 0, 1, 2
ivf_train = train[train["시술 유형"] == "IVF"].copy()
for mapping_value in [0, 1, 2]:
    subset = ivf_train[ivf_train["시술유형매핑"] == mapping_value].copy()
    if len(subset) > 0:
        X_ivf = subset[features].values
        y_ivf = subset[target].values
        X_train_ivf, X_val_ivf, y_train_ivf, y_val_ivf = train_test_split(X_ivf, y_ivf, test_size=0.2, stratify=y_ivf, random_state=42)
        
        model_ivf = build_model(input_dim=len(features), dropout_rate=0.3, l2_reg=0.001)
        early_stop_ivf = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True, verbose=1)
        
        print(f"Training IVF model for 시술유형매핑 = {mapping_value} ...")
        history_ivf = model_ivf.fit(X_train_ivf, y_train_ivf,
                                    validation_data=(X_val_ivf, y_val_ivf),
                                    epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stop_ivf], verbose=2)
        
        val_pred_ivf = model_ivf.predict(X_val_ivf).flatten()
        val_pred_ivf_bin = (val_pred_ivf >= 0.5).astype(int)
        acc_ivf = accuracy_score(y_val_ivf, val_pred_ivf_bin)
        auc_ivf = roc_auc_score(y_val_ivf, val_pred_ivf)
        f1_ivf = f1_score(y_val_ivf, val_pred_ivf_bin)
        ll_ivf = log_loss(y_val_ivf, val_pred_ivf)
        print(f"IVF group (시술유형매핑 = {mapping_value}) validation results:")
        print("  Accuracy: {:.4f}".format(acc_ivf))
        print("  AUC: {:.4f}".format(auc_ivf))
        print("  F1 Score: {:.4f}".format(f1_ivf))
        print("  Log Loss: {:.4f}".format(ll_ivf))
        
        models[f"IVF_{mapping_value}"] = model_ivf
        results[f"IVF_{mapping_value}"] = {"accuracy": acc_ivf, "auc": auc_ivf, "f1": f1_ivf, "log_loss": ll_ivf}

# =======================================
# 4. Test 데이터에 대한 예측 (딥러닝 모델 재사용)
# =======================================
# test 데이터에도 "임신 성공 여부" 컬럼에 예측값을 채웁니다.
test[target] = np.nan

# 예측 함수: 예측 확률을 반환한 후 0.5 임계값으로 이진화
def predict_dl(model, X):
    preds = model.predict(X).flatten()
    return (preds >= 0.5).astype(int)

# (A) DI 예측: "시술 유형"이 DI인 경우
di_test = test[test["시술 유형"] == "DI"].copy()
if len(di_test) > 0:
    X_di_test = di_test[features].values
    pred_di_test = predict_dl(models["DI"], X_di_test)
    test.loc[di_test.index, target] = pred_di_test

# (B) IVF 예측: "시술 유형"이 IVF인 경우 – 각 그룹별 "시술유형매핑"
ivf_test = test[test["시술 유형"] == "IVF"].copy()
if len(ivf_test) > 0:
    for mapping_value in [0, 1, 2]:
        idx = test[(test["시술 유형"] == "IVF") & (test["시술유형매핑"] == mapping_value)].index
        if len(idx) > 0 and f"IVF_{mapping_value}" in models:
            X_ivf_test = test.loc[idx, features].values
            pred_ivf_test = predict_dl(models[f"IVF_{mapping_value}"], X_ivf_test)
            test.loc[idx, target] = pred_ivf_test

# =======================================
# 5. 결과 저장 및 출력
# =======================================
test.to_csv("test_with_predictions_deeplearning.csv", index=False, encoding='cp949')
print("최종 예측 결과가 'test_with_predictions_deeplearning.csv' 파일에 저장되었습니다.")

# =======================================
# 6. Train 및 Test 결과 요약 출력
# =======================================
print("\n=== Train 그룹별 Validation 결과 ===")
for key, metrics in results.items():
    print(f"{key}: Accuracy={metrics['accuracy']:.4f}, AUC={metrics['auc']:.4f}, F1={metrics['f1']:.4f}, LogLoss={metrics['log_loss']:.4f}")

# test 데이터의 예측 분포 확인
print("\n=== Test 데이터 예측 분포 ===")
print(test[target].value_counts())

C:\Users\leoyo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training DI model...
Epoch 1/100
20/20 - 4s - 196ms/step - accuracy: 0.4877 - loss: 0.9736 - val_accuracy: 0.4948 - val_loss: 0.7599
Epoch 2/100
20/20 - 0s - 6ms/step - accuracy: 0.5302 - loss: 0.8626 - val_accuracy: 0.8777 - val_loss: 0.7450
Epoch 3/100
20/20 - 0s - 6ms/step - accuracy: 0.5855 - loss: 0.7802 - val_accuracy: 0.8769 - val_loss: 0.7234
Epoch 4/100
20/20 - 0s - 5ms/step - accuracy: 0.6212 - loss: 0.7280 - val_accuracy: 0.8769 - val_loss: 0.7033
Epoch 5/100
20/20 - 0s - 5ms/step - accuracy: 0.6707 - loss: 0.6931 - val_accuracy: 0.8769 - val_loss: 0.6775
Epoch 6/100
20/20 - 0s - 5ms/step - accuracy: 0.6979 - loss: 0.6632 - val_accuracy: 0.8769 - val_loss: 0.6544
Epoch 7/100
20/20 - 0s - 5ms/step - accuracy: 0.7357 - loss: 0.6350 - val_accuracy: 0.8721 - val_loss: 0.6212
Epoch 8/100
20/20 - 0s - 5ms/step - accuracy: 0.7689 - loss: 0.6025 - val_accuracy: 0.8721 - val_loss: 0.5957
Epoch 9/100
20/20 - 0s - 5ms/step - accuracy: 0.7945 - loss: 0.5768 - val_accuracy: 0.8721 - val_

C:\Users\leoyo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


401/401 - 5s - 11ms/step - accuracy: 0.7840 - loss: 0.5375 - val_accuracy: 0.7769 - val_loss: 0.5177
Epoch 2/100
401/401 - 1s - 2ms/step - accuracy: 0.8059 - loss: 0.4561 - val_accuracy: 0.8102 - val_loss: 0.4306
Epoch 3/100
401/401 - 1s - 2ms/step - accuracy: 0.8083 - loss: 0.4342 - val_accuracy: 0.8102 - val_loss: 0.4161
Epoch 4/100
401/401 - 1s - 2ms/step - accuracy: 0.8097 - loss: 0.4224 - val_accuracy: 0.8103 - val_loss: 0.4065
Epoch 5/100
401/401 - 1s - 2ms/step - accuracy: 0.8088 - loss: 0.4147 - val_accuracy: 0.8099 - val_loss: 0.4024
Epoch 6/100
401/401 - 1s - 2ms/step - accuracy: 0.8092 - loss: 0.4098 - val_accuracy: 0.8105 - val_loss: 0.4001
Epoch 7/100
401/401 - 1s - 2ms/step - accuracy: 0.8094 - loss: 0.4061 - val_accuracy: 0.8102 - val_loss: 0.3983
Epoch 8/100
401/401 - 1s - 2ms/step - accuracy: 0.8108 - loss: 0.4050 - val_accuracy: 0.8097 - val_loss: 0.3978
Epoch 9/100
401/401 - 1s - 2ms/step - accuracy: 0.8105 - loss: 0.4038 - val_accuracy: 0.8103 - val_loss: 0.3958
Epo

C:\Users\leoyo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


297/297 - 4s - 13ms/step - accuracy: 0.7253 - loss: 0.6074 - val_accuracy: 0.7682 - val_loss: 0.5913
Epoch 2/100
297/297 - 1s - 2ms/step - accuracy: 0.7969 - loss: 0.4768 - val_accuracy: 0.8035 - val_loss: 0.4587
Epoch 3/100
297/297 - 1s - 2ms/step - accuracy: 0.8008 - loss: 0.4529 - val_accuracy: 0.8083 - val_loss: 0.4291
Epoch 4/100
297/297 - 1s - 3ms/step - accuracy: 0.8056 - loss: 0.4372 - val_accuracy: 0.8061 - val_loss: 0.4208
Epoch 5/100
297/297 - 1s - 2ms/step - accuracy: 0.8051 - loss: 0.4277 - val_accuracy: 0.8087 - val_loss: 0.4134
Epoch 6/100
297/297 - 1s - 2ms/step - accuracy: 0.8052 - loss: 0.4213 - val_accuracy: 0.8065 - val_loss: 0.4079
Epoch 7/100
297/297 - 1s - 2ms/step - accuracy: 0.8052 - loss: 0.4149 - val_accuracy: 0.8077 - val_loss: 0.4050
Epoch 8/100
297/297 - 1s - 2ms/step - accuracy: 0.8051 - loss: 0.4109 - val_accuracy: 0.8071 - val_loss: 0.4001
Epoch 9/100
297/297 - 1s - 2ms/step - accuracy: 0.8044 - loss: 0.4075 - val_accuracy: 0.8076 - val_loss: 0.3964
Epo

C:\Users\leoyo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


85/85 - 4s - 47ms/step - accuracy: 0.5713 - loss: 0.8421 - val_accuracy: 0.7627 - val_loss: 0.6640
Epoch 2/100
85/85 - 0s - 3ms/step - accuracy: 0.6573 - loss: 0.6964 - val_accuracy: 0.7627 - val_loss: 0.6338
Epoch 3/100
85/85 - 0s - 2ms/step - accuracy: 0.7138 - loss: 0.6371 - val_accuracy: 0.7627 - val_loss: 0.6121
Epoch 4/100
85/85 - 0s - 2ms/step - accuracy: 0.7398 - loss: 0.6117 - val_accuracy: 0.7635 - val_loss: 0.5980
Epoch 5/100
85/85 - 0s - 3ms/step - accuracy: 0.7534 - loss: 0.5927 - val_accuracy: 0.7659 - val_loss: 0.5804
Epoch 6/100
85/85 - 0s - 3ms/step - accuracy: 0.7586 - loss: 0.5822 - val_accuracy: 0.7688 - val_loss: 0.5675
Epoch 7/100
85/85 - 0s - 4ms/step - accuracy: 0.7609 - loss: 0.5750 - val_accuracy: 0.7698 - val_loss: 0.5582
Epoch 8/100
85/85 - 0s - 3ms/step - accuracy: 0.7637 - loss: 0.5693 - val_accuracy: 0.7692 - val_loss: 0.5530
Epoch 9/100
85/85 - 0s - 3ms/step - accuracy: 0.7652 - loss: 0.5622 - val_accuracy: 0.7705 - val_loss: 0.5490
Epoch 10/100
85/85 - 

In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, log_loss

# =======================================
# 1. 데이터 불러오기 및 기본 설정
# =======================================
train = pd.read_csv('train_data_진짜.csv', encoding='cp949')
test = pd.read_csv('test_data_진짜.csv', encoding='cp949')

# 전체 인풋 피처 (4개)
features_all = ["난자 점수", "정자 점수", "나이 점수", "시술점수"]
target = "임신 성공 여부"

# train, test에는 이미 "시술 유형"과 "시술유형매핑" 칼럼이 존재한다고 가정합니다.
# 그룹: DI, IVF_0, IVF_1, IVF_2
groups = {}
groups["DI"] = train[train["시술 유형"] == "DI"].copy()
ivf = train[train["시술 유형"] == "IVF"].copy()
for mapping_value in [0, 1, 2]:
    groups[f"IVF_{mapping_value}"] = ivf[ivf["시술유형매핑"] == mapping_value].copy()

# =======================================
# 2. 인풋 피처 조합 생성 함수
# =======================================
def get_feature_subsets(features):
    """
    전체 4개 피처("all")와, 각 1개씩 제외한 3개 피처 조합을 딕셔너리로 반환합니다.
    """
    subsets = {}
    subsets["all"] = features
    for feat in features:
        subset = [f for f in features if f != feat]
        subsets[f"drop_{feat}"] = subset
    return subsets

feature_subsets = get_feature_subsets(features_all)
# 예: keys: "all", "drop_난자 점수", "drop_정자 점수", "drop_나이 점수", "drop_시술점수"

# =======================================
# 3. 딥러닝 모델 구축 함수
# =======================================
def build_model(input_dim, dropout_rate=0.3, l2_reg=0.001):
    model = Sequential()
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(l2_reg), input_dim=input_dim))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(16, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# =======================================
# 4. 각 그룹별로 전체 모델(모든 조합) 학습 함수
# =======================================
def train_group_models(group_data, feature_subsets, target, epochs=100, batch_size=256, patience=10):
    """
    group_data: 해당 그룹의 DataFrame
    feature_subsets: 딕셔너리, key: subset 이름, value: list of 피처 이름들
    target: 타겟 칼럼 이름
    ---
    반환: dictionary {subset_key: trained model}와 validation metrics dictionary
    """
    X_full = group_data[features_all]
    y = group_data[target].astype(int).values
    # train/validation split (80:20, stratified)
    X_train_full, X_val_full, y_train, y_val = train_test_split(X_full, y, test_size=0.2, stratify=y, random_state=42)
    
    models_dict = {}
    val_preds_list = []
    
    for subset_key, feat_list in feature_subsets.items():
        print(f"Training model with features: {feat_list} (subset: {subset_key})")
        input_dim = len(feat_list)
        model = build_model(input_dim=input_dim, dropout_rate=0.3, l2_reg=0.001)
        early_stop = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, verbose=1)
        
        model.fit(X_train_full[feat_list].values, y_train,
                  validation_data=(X_val_full[feat_list].values, y_val),
                  epochs=epochs, batch_size=batch_size, callbacks=[early_stop], verbose=2)
        models_dict[subset_key] = model
        
        # 예측 on validation for ensemble (store probability predictions)
        pred_val = model.predict(X_val_full[feat_list].values).flatten()
        val_preds_list.append(pred_val)
    
    # Ensemble: 평균
    val_preds_ensemble = np.mean(np.array(val_preds_list), axis=0)
    val_pred_bin = (val_preds_ensemble >= 0.5).astype(int)
    
    acc = accuracy_score(y_val, val_pred_bin)
    auc = roc_auc_score(y_val, val_preds_ensemble)
    f1 = f1_score(y_val, val_pred_bin)
    ll = log_loss(y_val, val_preds_ensemble)
    metrics = {"accuracy": acc, "auc": auc, "f1": f1, "log_loss": ll}
    
    print(f"Validation metrics: Accuracy={acc:.4f}, AUC={auc:.4f}, F1={f1:.4f}, LogLoss={ll:.4f}")
    
    return models_dict, metrics

# =======================================
# 5. Train 데이터 그룹별 모델 학습 (전체 조합 앙상블)
# =======================================
group_models = {}    # key: group (e.g., "DI", "IVF_0", ...)
group_metrics = {}   # validation metrics per group

for group_key, data in groups.items():
    print(f"\n=== Training models for group: {group_key} ===")
    if len(data) == 0:
        continue
    models_dict, metrics = train_group_models(data, feature_subsets, target,
                                                epochs=100, batch_size=256, patience=10)
    group_models[group_key] = models_dict
    group_metrics[group_key] = metrics

# =======================================
# 6. Test 데이터에 대한 예측 (각 그룹별 앙상블)
# =======================================
# test 데이터에도 "임신 성공 여부"를 채울 것입니다.
test[target] = np.nan

def ensemble_predict(models_dict, X, feature_subsets):
    """
    models_dict: dictionary {subset_key: trained model}
    X: DataFrame of test data (전체 피처 포함)
    feature_subsets: dictionary of feature subsets (same keys as models_dict)
    반환: ensemble prediction (binary vector) using simple average thresholded at 0.5.
    """
    preds = []
    for subset_key, model in models_dict.items():
        feat_list = feature_subsets[subset_key]
        pred = model.predict(X[feat_list].values).flatten()
        preds.append(pred)
    ensemble_prob = np.mean(np.array(preds), axis=0)
    return (ensemble_prob >= 0.5).astype(int)

# DI 그룹 예측
di_test = test[test["시술 유형"] == "DI"].copy()
if len(di_test) > 0:
    pred_di = ensemble_predict(group_models["DI"], di_test, feature_subsets)
    test.loc[di_test.index, target] = pred_di

# IVF 그룹 예측 (시술유형매핑별: 0, 1, 2)
for mapping_value in [0, 1, 2]:
    group_key = f"IVF_{mapping_value}"
    ivf_subset = test[(test["시술 유형"] == "IVF") & (test["시술유형매핑"] == mapping_value)].copy()
    if len(ivf_subset) > 0 and group_key in group_models:
        pred_ivf = ensemble_predict(group_models[group_key], ivf_subset, feature_subsets)
        test.loc[ivf_subset.index, target] = pred_ivf

# =======================================
# 7. 결과 저장 및 출력
# =======================================
test.to_csv("test_with_predictions_ensemble.csv", index=False, encoding='cp949')
print("최종 예측 결과가 'test_with_predictions_ensemble.csv' 파일에 저장되었습니다.")

print("\n=== Train 그룹별 Validation 결과 ===")
for group_key, metrics in group_metrics.items():
    print(f"{group_key}: Accuracy={metrics['accuracy']:.4f}, AUC={metrics['auc']:.4f}, F1={metrics['f1']:.4f}, LogLoss={metrics['log_loss']:.4f}")

print("\n=== Test 데이터 예측 분포 ===")
print(test[target].value_counts())



=== Training models for group: DI ===
Training model with features: ['난자 점수', '정자 점수', '나이 점수', '시술점수'] (subset: all)
Epoch 1/100


C:\Users\leoyo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 - 4s - 176ms/step - accuracy: 0.4990 - loss: 0.9550 - val_accuracy: 0.8713 - val_loss: 0.7188
Epoch 2/100
20/20 - 0s - 21ms/step - accuracy: 0.5634 - loss: 0.8272 - val_accuracy: 0.8713 - val_loss: 0.6858
Epoch 3/100
20/20 - 0s - 5ms/step - accuracy: 0.6415 - loss: 0.7480 - val_accuracy: 0.8713 - val_loss: 0.6547
Epoch 4/100
20/20 - 0s - 5ms/step - accuracy: 0.6763 - loss: 0.7120 - val_accuracy: 0.8713 - val_loss: 0.6258
Epoch 5/100
20/20 - 0s - 4ms/step - accuracy: 0.7216 - loss: 0.6830 - val_accuracy: 0.8713 - val_loss: 0.6008
Epoch 6/100
20/20 - 0s - 5ms/step - accuracy: 0.7562 - loss: 0.6374 - val_accuracy: 0.8713 - val_loss: 0.5768
Epoch 7/100
20/20 - 0s - 4ms/step - accuracy: 0.7826 - loss: 0.6227 - val_accuracy: 0.8713 - val_loss: 0.5555
Epoch 8/100
20/20 - 0s - 5ms/step - accuracy: 0.8045 - loss: 0.5935 - val_accuracy: 0.8713 - val_loss: 0.5414
Epoch 9/100
20/20 - 0s - 5ms/step - accuracy: 0.8215 - loss: 0.5676 - val_accuracy: 0.8713 - val_loss: 0.5257
Epoch 10/100
20/20 

C:\Users\leoyo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 - 4s - 202ms/step - accuracy: 0.5288 - loss: 0.8648 - val_accuracy: 0.8721 - val_loss: 0.7251
Epoch 2/100
20/20 - 0s - 5ms/step - accuracy: 0.5743 - loss: 0.7901 - val_accuracy: 0.8721 - val_loss: 0.6963
Epoch 3/100
20/20 - 0s - 4ms/step - accuracy: 0.6157 - loss: 0.7456 - val_accuracy: 0.8713 - val_loss: 0.6684
Epoch 4/100
20/20 - 0s - 5ms/step - accuracy: 0.6596 - loss: 0.7013 - val_accuracy: 0.8713 - val_loss: 0.6396
Epoch 5/100
20/20 - 0s - 5ms/step - accuracy: 0.7182 - loss: 0.6669 - val_accuracy: 0.8713 - val_loss: 0.6121
Epoch 6/100
20/20 - 0s - 5ms/step - accuracy: 0.7488 - loss: 0.6544 - val_accuracy: 0.8713 - val_loss: 0.5923
Epoch 7/100
20/20 - 0s - 9ms/step - accuracy: 0.7818 - loss: 0.6127 - val_accuracy: 0.8713 - val_loss: 0.5711
Epoch 8/100
20/20 - 0s - 6ms/step - accuracy: 0.8029 - loss: 0.5877 - val_accuracy: 0.8713 - val_loss: 0.5496
Epoch 9/100
20/20 - 0s - 5ms/step - accuracy: 0.8184 - loss: 0.5668 - val_accuracy: 0.8713 - val_loss: 0.5270
Epoch 10/100
20/20 -

...

85/85 - 4s - 52ms/step - accuracy: 0.5512 - loss: 0.7923 - val_accuracy: 0.7627 - val_loss: 0.6983
Epoch 2/100
85/85 - 0s - 3ms/step - accuracy: 0.6572 - loss: 0.6710 - val_accuracy: 0.7627 - val_loss: 0.6492
Epoch 3/100
85/85 - 0s - 3ms/step - accuracy: 0.7229 - loss: 0.6178 - val_accuracy: 0.7627 - val_loss: 0.6224
Epoch 4/100
85/85 - 0s - 4ms/step - accuracy: 0.7539 - loss: 0.5889 - val_accuracy: 0.7642 - val_loss: 0.5991
Epoch 5/100
85/85 - 0s - 3ms/step - accuracy: 0.7616 - loss: 0.5720 - val_accuracy: 0.7672 - val_loss: 0.5775
Epoch 6/100
85/85 - 0s - 3ms/step - accuracy: 0.7639 - loss: 0.5640 - val_accuracy: 0.7686 - val_loss: 0.5595
Epoch 7/100
85/85 - 0s - 3ms/step - accuracy: 0.7678 - loss: 0.5546 - val_accuracy: 0.7698 - val_loss: 0.5503
Epoch 8/100
85/85 - 0s - 3ms/step - accuracy: 0.7673 - loss: 0.5514 - val_accuracy: 0.7705 - val_loss: 0.5451
Epoch 9/100
85/85 - 0s - 3ms/step - accuracy: 0.7667 - loss: 0.5484 - val_accuracy: 0.7701 - val_loss: 0.5416
Epoch 10/100
85/85 - 

C:\Users\leoyo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


85/85 - 4s - 41ms/step - accuracy: 0.5730 - loss: 0.8659 - val_accuracy: 0.7627 - val_loss: 0.6268
Epoch 2/100
85/85 - 0s - 3ms/step - accuracy: 0.6664 - loss: 0.7008 - val_accuracy: 0.7627 - val_loss: 0.6072
Epoch 3/100
85/85 - 0s - 4ms/step - accuracy: 0.7188 - loss: 0.6434 - val_accuracy: 0.7627 - val_loss: 0.5958
Epoch 4/100
85/85 - 0s - 2ms/step - accuracy: 0.7405 - loss: 0.6182 - val_accuracy: 0.7631 - val_loss: 0.5861
Epoch 5/100
85/85 - 0s - 2ms/step - accuracy: 0.7516 - loss: 0.6016 - val_accuracy: 0.7660 - val_loss: 0.5753
Epoch 6/100
85/85 - 0s - 2ms/step - accuracy: 0.7554 - loss: 0.5902 - val_accuracy: 0.7688 - val_loss: 0.5657
Epoch 7/100
85/85 - 0s - 2ms/step - accuracy: 0.7583 - loss: 0.5812 - val_accuracy: 0.7701 - val_loss: 0.5578
Epoch 8/100
85/85 - 0s - 3ms/step - accuracy: 0.7629 - loss: 0.5715 - val_accuracy: 0.7703 - val_loss: 0.5525
Epoch 9/100
85/85 - 0s - 3ms/step - accuracy: 0.7620 - loss: 0.5666 - val_accuracy: 0.7709 - val_loss: 0.5482
Epoch 10/100
85/85 - 

C:\Users\leoyo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


85/85 - 4s - 42ms/step - accuracy: 0.5190 - loss: 0.8606 - val_accuracy: 0.7451 - val_loss: 0.7337
Epoch 2/100
85/85 - 0s - 3ms/step - accuracy: 0.6397 - loss: 0.7225 - val_accuracy: 0.7627 - val_loss: 0.6686
Epoch 3/100
85/85 - 0s - 3ms/step - accuracy: 0.7059 - loss: 0.6783 - val_accuracy: 0.7627 - val_loss: 0.6320
Epoch 4/100
85/85 - 0s - 3ms/step - accuracy: 0.7358 - loss: 0.6466 - val_accuracy: 0.7627 - val_loss: 0.6172
Epoch 5/100
85/85 - 0s - 3ms/step - accuracy: 0.7457 - loss: 0.6374 - val_accuracy: 0.7627 - val_loss: 0.6067
Epoch 6/100
85/85 - 0s - 3ms/step - accuracy: 0.7498 - loss: 0.6258 - val_accuracy: 0.7627 - val_loss: 0.6005
Epoch 7/100
85/85 - 0s - 3ms/step - accuracy: 0.7543 - loss: 0.6170 - val_accuracy: 0.7627 - val_loss: 0.5936
Epoch 8/100
85/85 - 0s - 3ms/step - accuracy: 0.7562 - loss: 0.6092 - val_accuracy: 0.7627 - val_loss: 0.5886
Epoch 9/100
85/85 - 0s - 3ms/step - accuracy: 0.7590 - loss: 0.6026 - val_accuracy: 0.7627 - val_loss: 0.5846
Epoch 10/100
85/85 - 